In [ ]:
# pip install hydroErr

In [ ]:
# pip install matplotlib

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import keras
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from keras import layers
from keras import initializers
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_blobs
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import HydroErr as he
from matplotlib import rcParams
import pickle
from sklearn.preprocessing import MinMaxScaler
import shap
import seaborn as sns
import re
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm

from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber

import gc
import tensorflow.keras.backend as K

In [ ]:
def compute_metrics(true_series, forecast):
    true_series = np.array(np.round(true_series, 12))
    forecast = np.array(np.round(forecast, 12))

    # R²
    R2 = 1 - ((np.sum((true_series - forecast) ** 2)) / (np.sum((true_series - np.mean(true_series)) ** 2)))

    # Nash-Sutcliffe Efficiency
    NSE = he.nse(true_series, forecast)

    # RMSE
    RMSE = np.sqrt(np.mean((true_series - forecast) ** 2))

    # Percent Bias (PBIAS)
    Pbias_num = np.sum(true_series - forecast)
    Pbias_denom = np.sum(true_series)
    Pbias = (Pbias_num / Pbias_denom) * 100

    return R2, NSE, RMSE, Pbias

In [ ]:
# Set font properties
rcParams.update({
    'font.size': 12,          # Default text size
    # 'font.family': 'serif',   # Font family
    # 'font.serif': ['Cambria'],  # Specific serif font  ['Times New Roman', Palatino', 'Garamond', 'Cambria', 'Bookman']
    'axes.titlesize': 14,     # Title font size
    'axes.labelsize': 12,     # X and Y label size
    'xtick.labelsize': 11,    # X tick label size
    'ytick.labelsize': 11,    # Y tick label size
    'legend.fontsize': 11,    # Legend font size
    'figure.titlesize': 16,   # Figure title size
    'figure.titleweight': 'bold'  # Figure title weight

}),

# function to plot time series and monthly averages with metrics
def plot_series(time, y_test, y_pred, format="-", start=0, end=None, title=None):

    # Plot the true and predicted values
    plt.figure(figsize=(12, 6))
    plt.plot(time[start:end], y_test[start:end], format, linewidth=1, label="True Values")
    plt.plot(time[start:end], y_pred[start:end], format, linewidth=1, label="Predicted Values")
    plt.title(f"Daily Average Streamflow - {title}", )
    plt.xlabel("Date",  style='italic')
    plt.ylabel("Streamflow (m$^3$/s)",  style='italic')
    # Compute metrics for the main plot and display them
    R2i, NSEi, RMSEi, Pbiasi = compute_metrics(y_test, y_pred)
    print(f"Metrics for Daily Average Streamflow - {title}:")
    print(f"  R2: {R2i:.5f}, NSE: {NSEi:.5f}, RMSE: {RMSEi:.5f}, Pbias: {Pbiasi:.5f}")
    # Add metrics as text on the plot
    textstr = '\n'.join((
        f"R$^2$: {R2i:.3f}",
        f"NSE: {NSEi:.3f}",
        f"RMSE: {RMSEi:.3f} m$^3$/s",
        f"Pbias: {Pbiasi:.3f} %"
    ))
    plt.gca().text(0.02, 0.96, textstr, transform=plt.gca().transAxes,
                   fontsize=11, verticalalignment='top', bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='black', alpha=0.7),
                   )
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

    # Compute monthly averages
    df = pd.DataFrame({'time': time, 'y_test': y_test, 'y_pred': y_pred})
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    monthly_avg = df.resample('ME').mean()

    # Plot the averaged monthly streamflow
    plt.figure(figsize=(12, 6))
    plt.plot(monthly_avg.index, monthly_avg['y_test'], label="True Values", linewidth=1, marker='o')
    plt.plot(monthly_avg.index, monthly_avg['y_pred'], label="Predicted Values", linewidth=1, marker='o')
    plt.title(f"Monthly Average Streamflow - {title}", )
    plt.xlabel("Month",  style='italic')
    plt.ylabel("Streamflow (m$^3$/s)",  style='italic')

    # Compute metrics for monthly averages
    R2m, NSEm, RMSEm, Pbiasm = compute_metrics(monthly_avg['y_test'], monthly_avg['y_pred'])
    print(f"Metrics for Monthly Average Streamflow - {title}:")
    print(f"  R2: {R2m:.5f}, NSE: {NSEm:.5f}, RMSE: {RMSEm:.5f}, Pbias: {Pbiasm:.5f}")

    # Add metrics for monthly averages as text on the plot
    textstr_monthly = '\n'.join((
        f"R$^2$: {R2m:.3f}",
        f"NSE: {NSEm:.3f}",
        f"RMSE: {RMSEm:.3f} m$^3$/s",
        f"Pbias: {Pbiasm:.3f} %"
    ))
    plt.gca().text(0.02, 0.96, textstr_monthly, transform=plt.gca().transAxes,
                   fontsize=11, verticalalignment='top', bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='black', alpha=0.7),
                   )
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

In [ ]:
# Enable interactive plotting in Jupyter Notebook
%matplotlib notebook
# Ensure inline plotting in Jupyter Notebook
%matplotlib inline

# Fetching Climate Data

### ERA5-Land Climate data

In [ ]:
# Folder where all the CSV files are stored
folder_path = "/***/***/Climate_data"  # Replace with the actual path to the file

# Initialize an empty list to hold each file's DataFrame
all_dataframes = []

# Function to safely convert date values to 'YYYY-MM-DD'
def convert_mixed_date(val):
    if pd.isna(val):
        return val
    val = str(val)
    if re.match(r"^\d{8}$", val):  # Format: YYYYMMDD
        return pd.to_datetime(val, format='%Y%m%d').strftime('%Y-%m-%d')
    try:
        return pd.to_datetime(val).strftime('%Y-%m-%d')
    except:
        return pd.NaT

# Loop through each CSV file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        # Construct the full path to the file
        file_path = os.path.join(folder_path, file_name)

        # Read the CSV into a DataFrame
        df = pd.read_csv(file_path)

        # Convert 'date' column format from mixed formats to YYYY-MM-DD if the 'date' column exists
        if 'date' in df.columns:
            df['date'] = df['date'].apply(convert_mixed_date)

        # Rename columns for consistency
        df.rename(columns={'date': 'Date'}, inplace=True)
        df.rename(columns={'station_no': 'Station No.'}, inplace=True)

        # Append the DataFrame to the list
        all_dataframes.append(df)

# Concatenate all the DataFrames into a single DataFrame
WEF_Model_Climate_data = pd.concat(all_dataframes, ignore_index=True)

# Sort the final DataFrame by Station No. and Date
WEF_Model_Climate_data = WEF_Model_Climate_data.sort_values(by=['Station No.', 'Date']).reset_index(drop=True)

# Optional: Remove unwanted columns if they exist
WEF_Model_Climate_data.drop(columns=['system:index', '.geo'], errors='ignore', inplace=True)

In [ ]:
WEF_Model_Climate_data

In [ ]:
# Optionally, to save it to a CSV file:
# WEF_Model_Climate_data.to_csv('WEF_Model_Climate_data.csv', index=False)

In [ ]:
Parameters_in_forecasted_dataset = [
    'Station No.',
    'Date',
    'u_component_of_wind_10m',                    # 10m U-Component of Wind
    'v_component_of_wind_10m',                    # 10m V-Component of Wind
    'dewpoint_temperature_2m',                    # 2m Dewpoint Temperature
    'temperature_2m',                             # 2m Temperature
    'total_evaporation',                          # Total Evaporation
    'runoff',                                     # Runoff
    'surface_latent_heat_flux',                   # Surface Latent Heat Flux
    'surface_net_solar_radiation',                # Surface Net Solar Radiation
    'surface_net_thermal_radiation',              # Surface Net Thermal Radiation
    'surface_runoff',                             # Surface Runoff
    'surface_sensible_heat_flux',                 # Surface Sensible Heat Flux
    'surface_solar_radiation_downwards',          # Surface Solar Radiation Downwards
    'surface_thermal_radiation_downwards',        # Surface Thermal Radiation Downwards
    'snow_depth',                                 # Snow Depth
    'snow_density',                               # Snow Density
    'soil_temperature_level_1',                   # Soil Temperature Level 1
    'sub_surface_runoff',                         # Sub-Surface Runoff
    'total_precipitation',                        # Total Precipitation
    'volumetric_soil_water_layer_1'               # Volumetric Soil Moisture (Layer 1)
]

In [ ]:
WEF_Model_Climate_data = WEF_Model_Climate_data[Parameters_in_forecasted_dataset]

In [ ]:
WEF_Model_Climate_data

### Climate Data in 2007

In [ ]:
WEF_Model_Climate_data = WEF_Model_Climate_data[WEF_Model_Climate_data['Date'] >= '2007-01-01']

In [ ]:
WEF_Model_Climate_data

# Fetching Streamflow Data

### Streamflow stations data

In [ ]:
streamflow_stations = pd.read_excel('/***/***/Streamflow_stations.xlsx') # Replace with the actual path to the file

In [ ]:
streamflow_stations

In [ ]:
WEF_Model_info_data_07 = pd.read_excel('/***/***/Streamflow_Stations_info.xlsx') # Replace with the actual path to the file

In [ ]:
WEF_Model_info_data_07

### Streamflow data

In [ ]:
dsi_2005_Q_Data = pd.read_csv('/***/***/dsi_2005_Q_Data.csv') # Replace with the actual path to the file
dsi_2006_Q_Data = pd.read_csv('/***/***/dsi_2006_Q_Data.csv') # Replace with the actual path to the file
dsi_2007_Q_Data = pd.read_csv('/***/***/dsi_2007_Q_Data.csv') # Replace with the actual path to the file
dsi_2008_Q_Data = pd.read_csv('/***/***/dsi_2008_Q_Data.csv') # Replace with the actual path to the file
dsi_2009_Q_Data = pd.read_csv('/***/***/dsi_2009_Q_Data.csv') # Replace with the actual path to the file
dsi_2010_Q_Data = pd.read_csv('/***/***/dsi_2010_Q_Data.csv') # Replace with the actual path to the file
dsi_2011_Q_Data = pd.read_csv('/***/***/dsi_2011_Q_Data.csv') # Replace with the actual path to the file
dsi_2012_Q_Data = pd.read_csv('/***/***/dsi_2012_Q_Data.csv') # Replace with the actual path to the file
dsi_2013_Q_Data = pd.read_csv('/***/***/dsi_2013_Q_Data.csv') # Replace with the actual path to the file
dsi_2014_Q_Data = pd.read_csv('/***/***/dsi_2014_Q_Data.csv') # Replace with the actual path to the file
dsi_2015_Q_Data = pd.read_csv('/***/***/dsi_2015_Q_Data.csv') # Replace with the actual path to the file
dsi_2016_Q_Data = pd.read_csv('/***/***/dsi_2016_Q_Data.csv') # Replace with the actual path to the file
dsi_2017_Q_Data = pd.read_csv('/***/***/dsi_2017_Q_Data.csv') # Replace with the actual path to the file
dsi_2018_Q_Data = pd.read_csv('/***/***/dsi_2018_Q_Data.csv') # Replace with the actual path to the file
dsi_2019_Q_Data = pd.read_csv('/***/***/dsi_2019_Q_Data.csv') # Replace with the actual path to the file
dsi_2020_Q_Data = pd.read_csv('/***/***/dsi_2020_Q_Data.csv') # Replace with the actual path to the file

In [ ]:
# List of DataFrames
dsi_Q_data = [
    dsi_2005_Q_Data, dsi_2006_Q_Data, dsi_2007_Q_Data, dsi_2008_Q_Data,
    dsi_2009_Q_Data, dsi_2010_Q_Data, dsi_2011_Q_Data, dsi_2012_Q_Data,
    dsi_2013_Q_Data, dsi_2014_Q_Data, dsi_2015_Q_Data, dsi_2016_Q_Data,
    dsi_2017_Q_Data, dsi_2018_Q_Data, dsi_2019_Q_Data, dsi_2020_Q_Data,
]

# Filter each DataFrame and store the result in a list
WEF_Model_streamflow_df = [df for df in dsi_Q_data]

# Concatenate the filtered DataFrames into a single DataFrame
WEF_Model_streamflow_df = pd.concat(WEF_Model_streamflow_df)

# Reset the index of the resulting DataFrame
WEF_Model_streamflow_df.reset_index(drop=True, inplace=True)
WEF_Model_streamflow_df['Date'] = pd.to_datetime(WEF_Model_streamflow_df['Date'], format='%d/%m/%Y')

WEF_Model_stations = WEF_Model_info_data_07['Station No.'].unique()
print(WEF_Model_stations)

WEF_Model_streamflow_df = WEF_Model_streamflow_df[WEF_Model_streamflow_df['Station No.'].isin(WEF_Model_stations)]

In [ ]:
dsi_Q_data

### Model stations data

### streamflow data in 2007

In [ ]:
WEF_Model_streamflow_data = WEF_Model_streamflow_df[WEF_Model_streamflow_df['Date'] >= '2007-01-01']

In [ ]:
WEF_Model_streamflow_data

In [ ]:
# Step 1: Check data types of all columns
print("Column Types Before Conversion:")
print(WEF_Model_streamflow_data.dtypes)

# Step 2: Identify non-numeric values before conversion using .loc for safe access
non_numeric_before_conversion = WEF_Model_streamflow_data.loc[~WEF_Model_streamflow_data['Flow rate (m3/s)'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notna()]

# Preview non-numeric values before conversion
print("Non-numeric values in 'Flow rate (m3/s)' column before conversion:")
print(non_numeric_before_conversion[['Station No.', 'Date', 'Flow rate (m3/s)']])

# Step 3: Convert 'Flow rate (m3/s)' column to numeric using .loc to avoid warnings
WEF_Model_streamflow_data.loc[:, 'Flow rate (m3/s)'] = pd.to_numeric(WEF_Model_streamflow_data['Flow rate (m3/s)'], errors='coerce')

# Step 4: Verify the conversion by displaying any NaN values in 'Flow rate (m3/s)'
non_convertible_after_conversion = WEF_Model_streamflow_data.loc[WEF_Model_streamflow_data['Flow rate (m3/s)'].isna(), ['Station No.', 'Date', 'Flow rate (m3/s)']]

# Display non-convertible entries after conversion to confirm they match the previewed non-numeric values
print("\nNon-convertible values in 'Flow rate (m3/s)' column after conversion to NaN:")
print(non_convertible_after_conversion)

In [ ]:
# Ensure 'Date' column is in datetime format
WEF_Model_streamflow_data.loc[:, 'Date'] = pd.to_datetime(WEF_Model_streamflow_data['Date'])
# List to collect the reindexed station DataFrames
reindexed_dfs = []
# Loop through each station and reindex its date range
for station in WEF_Model_streamflow_data['Station No.'].unique():
    station_df = WEF_Model_streamflow_data[WEF_Model_streamflow_data['Station No.'] == station].copy()
    # Create complete date range for the station
    full_date_range = pd.date_range(start=station_df['Date'].min(), end=station_df['Date'].max(), freq='D')
    # Set 'Date' as index for reindexing
    station_df = station_df.set_index('Date')
    # Reindex to include all dates (missing dates will have NaNs)
    station_df = station_df.reindex(full_date_range)
    # Restore 'Date' as a column
    station_df = station_df.rename_axis('Date').reset_index()
    # Refill station number where missing
    station_df['Station No.'] = station
    # Append to list
    reindexed_dfs.append(station_df)
# Concatenate all reindexed station DataFrames
WEF_Model_streamflow_data = pd.concat(reindexed_dfs).reset_index(drop=True)

In [ ]:
# Identify stations with NaNs before interpolation
stations_with_nans = WEF_Model_streamflow_data[WEF_Model_streamflow_data['Flow rate (m3/s)'].isna()]['Station No.'].unique()

print(len(stations_with_nans))
print(stations_with_nans)

In [ ]:
# Ensure 'Date' column is datetime
WEF_Model_streamflow_data.loc[:, 'Date'] = pd.to_datetime(WEF_Model_streamflow_data['Date'])
WEF_Model_streamflow_data = WEF_Model_streamflow_data.sort_values(['Station No.', 'Date']).reset_index(drop=True)

# Function to identify missing periods
def identify_missing_periods(data, station):
    missing_intervals = []
    total_missing_days = 0
    data = data.sort_values('Date').reset_index(drop=True)
    missing_data = data[data['Flow rate (m3/s)'].isna()]
    if missing_data.empty:
        return missing_intervals
    start_date = missing_data.iloc[0]['Date']
    previous_date = start_date

    for current_date in missing_data['Date'].iloc[1:]:
        if (current_date - previous_date).days == 1:
            previous_date = current_date
        else:
            missing_intervals.append((start_date, previous_date, (previous_date - start_date).days + 1))
            total_missing_days += (previous_date - start_date).days + 1
            start_date = current_date
            previous_date = current_date

    missing_intervals.append((start_date, previous_date, (previous_date - start_date).days + 1))
    total_missing_days += (previous_date - start_date).days + 1
    return missing_intervals

# Function to interpolate gaps with ≤ 31 missing days
def interpolate_small_gaps(data, station):
    data = data.copy()
    data = data.sort_values('Date').reset_index(drop=True)
    data['Flow rate (m3/s)'] = pd.to_numeric(data['Flow rate (m3/s)'], errors='coerce')
    missing_intervals = identify_missing_periods(data, station)
    small_gaps = [interval for interval in missing_intervals if interval[2] <= 31]

    for start, end, days in small_gaps:
        print(f"\nInterpolating for Station {station}: {start.date()} to {end.date()} ({days} days)")
        mask = (data['Date'] >= start) & (data['Date'] <= end)

        before = data[data['Date'] < start].dropna(subset=['Flow rate (m3/s)'])
        after = data[data['Date'] > end].dropna(subset=['Flow rate (m3/s)'])
        before_val = before.iloc[-1]['Flow rate (m3/s)'] if not before.empty else np.nan
        after_val = after.iloc[0]['Flow rate (m3/s)'] if not after.empty else np.nan

        if pd.notna(before_val) and pd.notna(after_val):
            data.loc[mask, 'Flow rate (m3/s)'] = data['Flow rate (m3/s)'].interpolate(method='linear')
        elif pd.isna(before_val) and pd.notna(after_val):
            print(f"⚠️ Gap at the start of the dataset. Forward filling for {station}.")
            data.loc[mask, 'Flow rate (m3/s)'] = data['Flow rate (m3/s)'].bfill()
        elif pd.notna(before_val) and pd.isna(after_val):
            print(f"⚠️ Gap at the end of the dataset. Backward filling for {station}.")
            data.loc[mask, 'Flow rate (m3/s)'] = data['Flow rate (m3/s)'].ffill()
        else:
            print(f"⚠️ Skipping gap due to insufficient data for interpolation at Station {station}")

    return data

# Display stations with remaining NaNs
def display_stations_with_remaining_nans(data):
    stations_with_nans = []
    for station in data['Station No.'].unique():
        station_data = data[data['Station No.'] == station]
        remaining_nans = identify_missing_periods(station_data, station)
        if remaining_nans:
            stations_with_nans.append(station)
            print(f"\n⚠️ Station with Remaining NaNs: {station}")
            for start, end, days in remaining_nans:
                print(f"  Remaining NaN period: {start.date()} to {end.date()}, Days missing: {days}")

    if not stations_with_nans:
        print("\n✅ No stations have remaining NaN values after interpolation.")
    else:
        print(f"\n⚠️ Total stations with remaining NaN values: {len(stations_with_nans)}")

# Main interpolation function by period
def run_interpolation(WEF_Model_streamflow_data, start_date_threshold, end_date_threshold, period_name):
    print(f"\n🔧 Running interpolation for {period_name} period: {start_date_threshold.date()} to {end_date_threshold.date()}")
    for station in tqdm(WEF_Model_streamflow_data['Station No.'].unique(), desc=f"Interpolating {period_name}"):
        full_station_data = WEF_Model_streamflow_data[WEF_Model_streamflow_data['Station No.'] == station]
        target_slice = full_station_data[(full_station_data['Date'] >= start_date_threshold) & (full_station_data['Date'] <= end_date_threshold)]

        if target_slice.empty:
            continue

        print(f"\n⛏️ Station {station} - Initial missing periods:")
        for start, end, days in identify_missing_periods(target_slice, station):
            print(f"  {start.date()} to {end.date()}, {days} days")

        interpolated_slice = interpolate_small_gaps(target_slice, station)

        # Replace only the slice being processed
        mask_to_remove = (
            (WEF_Model_streamflow_data['Station No.'] == station) &
            (WEF_Model_streamflow_data['Date'] >= start_date_threshold) &
            (WEF_Model_streamflow_data['Date'] <= end_date_threshold)
        )
        WEF_Model_streamflow_data = WEF_Model_streamflow_data[~mask_to_remove]
        WEF_Model_streamflow_data = pd.concat([WEF_Model_streamflow_data, interpolated_slice])

    # Re-sort after all processing
    WEF_Model_streamflow_data = WEF_Model_streamflow_data.sort_values(['Station No.', 'Date']).reset_index(drop=True)

    print(f"\n✅ Interpolation complete for {period_name} period.")
    display_stations_with_remaining_nans(WEF_Model_streamflow_data)
    return WEF_Model_streamflow_data

# === RUN INTERPOLATION FOR TRAINING PERIOD ===
WEF_Model_streamflow_data = run_interpolation(
    WEF_Model_streamflow_data,
    pd.Timestamp("2007-01-01"),
    pd.Timestamp("2017-09-30"),
    "training"
)

# === RUN INTERPOLATION FOR TESTING PERIOD ===
WEF_Model_streamflow_data = run_interpolation(
    WEF_Model_streamflow_data,
    pd.Timestamp("2017-10-01"),
    pd.Timestamp("2020-09-30"),
    "testing"
)

In [ ]:
# Identify stations with remaining NaNs after interpolation
stations_with_nans = WEF_Model_streamflow_data[WEF_Model_streamflow_data['Flow rate (m3/s)'].isna()]['Station No.'].unique()

print(len(stations_with_nans))
print(stations_with_nans)

In [ ]:
# Get unique station numbers
stations = WEF_Model_streamflow_data['Station No.'].unique()
print(f'No. of Stations:')
print(len(stations))
print(stations)

In [ ]:
len(WEF_Model_streamflow_data['Station No.'].unique())

In [ ]:
len(WEF_Model_Climate_data['Station No.'].unique())

In [ ]:
# Stations without downloaded climate data

streamflow_only_stations = np.setdiff1d(
    WEF_Model_streamflow_data['Station No.'].unique(),
    WEF_Model_Climate_data['Station No.'].unique()
)

print(streamflow_only_stations)

In [ ]:
# Checking the data type of the column 'Flow rate (m3/s)'
streamflow_dtype = WEF_Model_streamflow_data['Flow rate (m3/s)'].dtype

# Displaying the data type
streamflow_dtype

In [ ]:
# Identify non-numeric entries
non_numeric_entries = WEF_Model_streamflow_data[pd.to_numeric(WEF_Model_streamflow_data['Flow rate (m3/s)'], errors='coerce').isna()]
print(non_numeric_entries)

In [ ]:
WEF_Model_streamflow_data['Flow rate (m3/s)'] = pd.to_numeric(WEF_Model_streamflow_data['Flow rate (m3/s)'])

# Fetching S/E Data

### General

In [ ]:
regions_codes = {
     'Edirne': 'TR212',
     'Kırklareli': 'TR213',
     'Tekirdağ': 'TR211',
     'İstanbul': 'TR100',
     'Kocaeli': 'TR421',
     'Çanakkale': 'TR222',
     'Bursa': 'TR411',
     'Yalova': 'TR425',
     'Balıkesir': 'TR221',
     'Kütahya': 'TR333',
     'Unknown': 'Unknown',
     'Bilecik': 'TR413',
     'İzmir': 'TR310',
     'Manisa': 'TR331',
     'Uşak': 'TR334',
     'Aydın': 'TR321',
     'Denizli': 'TR322',
     'Afyonkarahisar': 'TR332',
     'Muğla': 'TR323',
     'Burdur': 'TR613',
     'Antalya': 'TR611',
     'Isparta': 'TR612',
     'Konya': 'TR521',
     'Ankara': 'TR510',
     'Eskişehir': 'TR412',
     'Sakarya': 'TR422',
     'Bolu': 'TR424',
     'Düzce': 'TR423',
     'Bartın': 'TR813',
     'Kastamonu': 'TR821',
     'Çankırı': 'TR822',
     'Karabük': 'TR812',
     'Zonguldak': 'TR811',
     'Sinop': 'TR823',
     'Samsun': 'TR831',
     'Tunceli': 'TRB14',
     'Amasya': 'TR834',
     'Gümüşhane': 'TR906',
     'Çorum': 'TR833',
     'Tokat': 'TR832',
     'Yozgat': 'TR723',
     'Ordu': 'TR902',
     'Erzincan': 'TRA12',
     'Sivas': 'TR722',
     'Giresun': 'TR903',
     'Niğde': 'TR713',
     'Kayseri': 'TR721',
     'Kırıkkale': 'TR711',
     'Nevşehir': 'TR714',
     'Kırşehir': 'TR715',
     'Aksaray': 'TR712',
     'Karaman': 'TR522',
     'Mersin': 'TR622',
     'Adana': 'TR621',
     'Kilis': 'TRC13',
     'Gaziantep': 'TRC11',
     'Hatay': 'TR631',
     'Kahramanmaraş': 'TR632',
     'Osmaniye': 'TR633',
     'Adıyaman': 'TRC12',
     'Erzurum': 'TRA11',
     'Ağrı': 'TRA21',
     'Bingöl': 'TRB13',
     'Malatya': 'TRB11',
     'Elazığ': 'TRB12',
     'Şanlıurfa': 'TRC21',
     'Muş': 'TRB22',
     'Diyarbakır': 'TRC22',
     'Mardin': 'TRC31',
     'Rize': 'TR904',
     'Trabzon': 'TR901',
     'Artvin': 'TR905',
     'Bayburt': 'TRA13',
     'Kars': 'TRA22',
     'Ardahan': 'TRA24',
     'Iğdır': 'TRA23',
     'Van': 'TRB21',
     'Bitlis': 'TRB23',
     'Hakkari': 'TRB24',
     'Şırnak': 'TRC33',
     'Siirt': 'TRC34',
     'Batman': 'TRC32'
}

# Invert the regions_codes dictionary to map NUTS_ID to City names
inverted_regions_codes = {v: k for k, v in regions_codes.items()}

In [ ]:
SE_codes = {
    "SE_01": "SE_01: Average size of households",
    "SE_02": "SE_02: Number of households by type Total",
    "SE_03": "SE_03: Number of districts",
    "SE_04": "SE_04: Number of municipalities",
    "SE_05": "SE_05: Number of villages",
    "SE_06": "SE_06: Total population",
    "SE_07": "SE_07: Population density (capita/square km)",
    "SE_08": "SE_08: Province and district centers population",
    "SE_09": "SE_09: Towns and villages population",

    "SE_10": "SE_10: Agricultural equipment and machinery Water pump",
    "SE_11": "SE_11: Animal products milk (tons)",
    "SE_12": "SE_12: Total areas for land under protective cover by type (decare)",
    "SE_13": "SE_13: Total harvested area of cereals and other crops (hectares)",
    "SE_14": "SE_14: Total arable land (hectare)",
    "SE_15": "SE_15: Total land under permanent crops (hectare)",
    "SE_16": "SE_16: Livestock Poultries (head)",
    "SE_17": "SE_17: Livestock (head)",
    "SE_18": "SE_18: Organic crop production area (transition period included) (Hectare)",
    "SE_19": "SE_19: Organic crop production amount (transition period included) (Tons)",
    "SE_20": "SE_20: Total production of area of cereals and other crops (tons)",
    "SE_21": "SE_21: Value of animal products (1000 TL)",
    "SE_22": "SE_22: Value of crop production (1000 TL)",
    "SE_23": "SE_23: Value of livestock (1000 TL)",
    "SE_24": "SE_24: Electricity consumption by Irrigation (MWh)",
    "SE_25": "SE_25: GDP by Agriculture (1000 TRY)",
    "SE_26": "SE_26: Number of Agriculture, forestry and fishery enterprises",
    "SE_27": "SE_27: Exports value by Agriculture, forestry and fishing (1000 USD)",
    "SE_28": "SE_28: Imports value by Agriculture, forestry and fishing (1000 USD)",

    "SE_29": "SE_29: Fresh ground water abstraction for municipal water supply network (1000 cubic meters/year)",
    "SE_30": "SE_30: Fresh surface water abstraction for municipal water supply network (1000 cubic meters/years)",
    "SE_31": "SE_31: Population served by drinking water treatment plants in total municipal population",
    "SE_32": "SE_32: Population served by water supply network in total municipal population",
    "SE_33": "SE_33: Total investments by municipalities in water supply facilities (TL)",
    "SE_34": "SE_34: Water abstraction per capita in municipalities (liters/capita-day)",
    "SE_35": "SE_35: Water treated by municipalities (1000 cubic meters/year)",
    "SE_36": "SE_36: Number of Water supply, sewerage, waste management and treatment activities enterprises",
    "SE_37": "SE_37: Exports value by Water supply; sewerage waste management and remediation activities (1000 USD)",
    "SE_38": "SE_38: Imports value by Water supply; sewerage waste management and remediation activities (1000 USD)",

    "SE_39": "SE_39: Electricity consumption of dwellings per capita (KWh)",
    "SE_40": "SE_40: Electricity consumption of industrial establishments per capita (KWh)",
    "SE_41": "SE_41: Total electricity consumptions (MWh)",
    "SE_42": "SE_42: Total electricity consumption per capita (KWh)",
    "SE_43": "SE_43: Gross Domestic Product (1000 TRY)",
    "SE_44": "SE_44: GDP by Industry (1000 TRY)",
    "SE_45": "SE_45: Per capita GDP ($)",
    "SE_46": "SE_46: Number of Manufacturing enterprises",
    "SE_47": "SE_47: Number of Mining and quarrying enterprises",
    "SE_48": "SE_48: Number of Accommodation and food service activities enterprises",
    "SE_49": "SE_49: Number of Construction enterprises",
    "SE_50": "SE_50: Number of Culture, arts, entertainment, recreation and sports enterprises",
    "SE_51": "SE_51: Number of Eklekrtik, gas, steam and air conditioning production and distribution enterprises",
    "SE_52": "SE_52: Total number of enterprises",
    "SE_53": "SE_53: Exports value by Electricity, gas, steam and air conditioning supply (1000 USD)",
    "SE_54": "SE_54: General total exports value (1000 USD)",
    "SE_55": "SE_55: Exports value by Manufacturing (1000 USD)",
    "SE_56": "SE_56: Exports value by Mining and quarrying (1000 USD)",
    "SE_57": "SE_57: Imports value by Electricity, gas, steam and air conditioning supply (1000 USD)",
    "SE_58": "SE_58: Imports value by General total (1000 USD)",
    "SE_59": "SE_59: Imports value by Manufacturing (1000 USD)",
    "SE_60": "SE_60: Imports value by Mining and quarrying (1000 USD)"
}

In [ ]:
SE_data = pd.read_csv('/***/***/SE_data.csv') # Replace with the actual path to the file

In [ ]:
SE_data = SE_data.sort_values(by=["NUTS_ID", "Year"])

In [ ]:
SE_data

In [ ]:
SE_data.columns

In [ ]:
intervals = {}

# Loop through each unique "NUTS_ID" and each "SE" column to identify the intervals and missing years
for nuts_id in SE_data['NUTS_ID'].unique():
    intervals[nuts_id] = {}
    for column in SE_data.columns:
        if column.startswith('SE'):
            # Filter data for the current NUTS_ID and column
            non_null_years = SE_data[(SE_data['NUTS_ID'] == nuts_id) & SE_data[column].notnull()]['Year']

            # Check if there are any non-null years
            if not non_null_years.empty:
                start_year = non_null_years.min()
                end_year = non_null_years.max()
                full_years = set(range(start_year, end_year + 1))
                available_years = set(non_null_years)
                missing_years = sorted(full_years - available_years)

                # Store the results for each NUTS_ID and SE column
                intervals[nuts_id][column] = {
                    'Start Year': start_year,
                    'End Year': end_year,
                    'Missing Years': missing_years
                }

# Convert the dictionary into a DataFrame with MultiIndex for better readability
intervals_df = pd.concat({k: pd.DataFrame(v).T for k, v in intervals.items()}, axis=0)
intervals_df.index.names = ['NUTS_ID', 'SE']
intervals_df.reset_index(inplace=True)

In [ ]:
# Display all rows
pd.set_option('display.max_rows', None)
# Print the DataFrame
print(intervals_df)
# Optionally, reset to default after printing (if you don't want this setting to affect future outputs)
pd.reset_option('display.max_rows')

In [ ]:
# Print the DataFrame
intervals_df

In [ ]:
# Filter only the parameters you're interested in
selected_SEs = [
    "SE06", "SE07", "SE08", "SE10", "SE13", "SE14", "SE15",
    "SE17", "SE29", "SE32", "SE34", "SE39", "SE40", "SE45"
]

filtered_df = intervals_df[intervals_df["SE"].isin(selected_SEs)]

# Group by SE and Start Year, then aggregate the NUTS_IDs under each
selected_SEs_av = (
    filtered_df
    .groupby(["SE", "Start Year"])["NUTS_ID"]
    .apply(list)
    .reset_index()
)

# Rename columns for clarity
selected_SEs_av.columns = ["SE", "Start Year", "NUTS_IDs"]
selected_SEs_av

### Interpolating missing S/E values

In [ ]:
# List to store columns with missing years
SE_Missing_Years = []

# Loop through each row in intervals_df
for index, row in intervals_df.iterrows():
    # Check if the SE column name starts with "SE"
    if row['SE'].startswith('SE'):
        # Check if there are any missing years
        if row["Missing Years"]:
            SE_Missing_Years.append(row['SE'])
            print(f"Missing years in {row['SE']} for NUTS_ID {row['NUTS_ID']}: {row['Missing Years']}")

In [ ]:
for nuts_id in SE_data['NUTS_ID'].unique():
    # Filter SE_data for the current NUTS_ID
    nuts_data = SE_data[SE_data['NUTS_ID'] == nuts_id]

    for column in SE_Missing_Years:
        # Ensure the column exists in SE_data
        if column in nuts_data.columns:
            # Retrieve the missing years information for the current NUTS_ID and column
            missing_years = intervals_df[(intervals_df['NUTS_ID'] == nuts_id) & (intervals_df['SE'] == column)]["Missing Years"].values

            if missing_years.size > 0:
                # print(f"Missing years in {column} for NUTS_ID {nuts_id}:", missing_years[0])

                # Create masks to check for valid data before and after each NaN
                valid_mask_before = nuts_data[column].shift(1).notna()
                valid_mask_after = nuts_data[column].shift(-1).notna()

                # Interpolate the data
                interpolated = nuts_data[column].interpolate(method='linear', limit_direction='both')

                # Update only the NaN values with valid neighbors
                SE_data.loc[nuts_data.index, column] = nuts_data[column].where(
                    nuts_data[column].notna() | ~(valid_mask_before & valid_mask_after),
                    interpolated
                )

                # Display the interpolated data for this NUTS_ID and column
                # print(SE_data[SE_data['NUTS_ID'] == nuts_id][["Year", "NUTS_ID", column]])

In [ ]:
SE_data.loc[:, 'SE_31'] = (SE_data['SE_31'] / 100) * SE_data['SE_06']
SE_data.loc[:, 'SE_32'] = (SE_data['SE_32'] / 100) * SE_data['SE_06']
SE_data.loc[:, 'SE_34'] = (SE_data['SE_34'] / 100) * SE_data['SE_06']

In [ ]:
intervals = {}

# Loop through each unique "NUTS_ID" and each "SE" column to identify the intervals and missing years
for nuts_id in SE_data['NUTS_ID'].unique():
    intervals[nuts_id] = {}
    for column in SE_data.columns:
        if column.startswith('SE'):
            # Filter data for the current NUTS_ID and column
            non_null_years = SE_data[(SE_data['NUTS_ID'] == nuts_id) & SE_data[column].notnull()]['Year']

            # Check if there are any non-null years
            if not non_null_years.empty:
                start_year = non_null_years.min()
                end_year = non_null_years.max()
                full_years = set(range(start_year, end_year + 1))
                available_years = set(non_null_years)
                missing_years = sorted(full_years - available_years)

                # Store the results for each NUTS_ID and SE column
                intervals[nuts_id][column] = {
                    'Start Year': start_year,
                    'End Year': end_year,
                    'Missing Years': missing_years
                }

# Convert the dictionary into a DataFrame with MultiIndex for better readability
intervals_df = pd.concat({k: pd.DataFrame(v).T for k, v in intervals.items()}, axis=0)
intervals_df.index.names = ['NUTS_ID', 'SE']
intervals_df.reset_index(inplace=True)

In [ ]:
# Display all rows
pd.set_option('display.max_rows', None)
# Print the DataFrame
print(intervals_df)
# Optionally, reset to default after printing (if you don't want this setting to affect future outputs)
pd.reset_option('display.max_rows')

In [ ]:
# Print the DataFrame
intervals_df

### Eliminating cities outside Model area of interest

In [ ]:
WEF_Model_WSHEDS_SA = pd.read_excel('/***/***/StreamflowStationsWSHEDS_SA.xlsx')# Replace with the actual path to the file
WEF_Model_WSHEDS_SA = WEF_Model_WSHEDS_SA.drop("Station No.", axis=1)

WEF_Model_NUTS_IDs = set()

# Iterate over each row in WEF_Model_WSHEDS_SA (each station)
for index, station_row in WEF_Model_WSHEDS_SA.iterrows():
    # Filter columns in station_row where the value is greater than 0.10 to get the cities and their weights
    city_weights = station_row[station_row > 0.10].drop('Station No.', errors='ignore')

    # Add unique cities (NUTS_IDs) to the set
    WEF_Model_NUTS_IDs.update(city_weights.index)

# Convert the set of unique NUTS_IDs to a list
WEF_Model_NUTS_IDs = list(WEF_Model_NUTS_IDs)
print("Unique NUTS_IDs (cities) with weights > 0.10:", WEF_Model_NUTS_IDs)

In [ ]:
SE_data_ = SE_data[SE_data['NUTS_ID'].isin(WEF_Model_NUTS_IDs)]

In [ ]:
SE_data_

### Eliminating S/E indicators with missing data in the period of interest

In [ ]:
SE_Start_Year = 2007
SE_End_Year = 2021

# Dictionary to store filtered SE_data for each NUTS_ID
filtered_SE_data_by_nuts_id = {}

# Initialize a set to track the columns that exist in every city and start with 'SE'
WEF_Model_SE_IDs = None

for nuts_id in SE_data_['NUTS_ID'].unique():
    # Collect `SE` columns that meet the criteria for this specific NUTS_ID
    q_parameters = [
        param for param, details in intervals.get(nuts_id, {}).items()
        if details['Start Year'] <= SE_Start_Year and
           details['End Year'] >= SE_End_Year and
           not details['Missing Years']
    ]

    # Define the columns to keep, including the required Year and NUTS_ID columns
    columns_to_keep = ['Year', 'NUTS_ID'] + q_parameters

    # Filter SE_data for this specific NUTS_ID and include only the specified columns
    nuts_data = SE_data_[SE_data_['NUTS_ID'] == nuts_id][columns_to_keep]

    # Further filter for the specified year range
    nuts_data = nuts_data[(nuts_data['Year'] >= SE_Start_Year) & (nuts_data['Year'] <= SE_End_Year)]

    # Store the filtered data for this NUTS_ID
    filtered_SE_data_by_nuts_id[nuts_id] = nuts_data

    # Track common columns: initialize with the first city's "SE" columns, then intersect with each subsequent city
    se_columns = {col for col in nuts_data.columns if col.startswith("SE")}
    if WEF_Model_SE_IDs is None:
        WEF_Model_SE_IDs = se_columns
    else:
        WEF_Model_SE_IDs.intersection_update(se_columns)

WEF_Model_SE_IDs = sorted(WEF_Model_SE_IDs)
print("No. of WEF_Model SE:", len(WEF_Model_SE_IDs))
print("WEF_Model_SE_IDs:", WEF_Model_SE_IDs)

In [ ]:
# Create a list of tuples (code, name) for each SE indicator in WEF_Model_SE_IDs
SE_indicators_with_names = [(code, SE_codes.get(code, f"Unknown_{code}")) for code in WEF_Model_SE_IDs]

# Display the result
for code, name in SE_indicators_with_names:
    print(f"{code}: {name}")

In [ ]:
# Filter SE_data to include only the columns in WEF_Model_SE_IDs
SE_data__ = SE_data_[['Year', 'NUTS_ID'] + WEF_Model_SE_IDs]

In [ ]:
SE_data__

### Eliminating years outside Model period of interest

In [ ]:
SE_Start_Year = 2007
SE_End_Year = 2020

# Filter SE_data to include only the columns in WEF_Model_SE_IDs and keep only rows within the specified year range
SE_data___ = SE_data__[(SE_data__['Year'] >= SE_Start_Year) & (SE_data__['Year'] <= SE_End_Year)]

In [ ]:
SE_data___

In [ ]:
SE_data = SE_data___

### Selected SEs

In [ ]:
selected_SEs = [
    "Year", "NUTS_ID",
    "SE_06", "SE_07", "SE_08", "SE_10", "SE_13", "SE_14", "SE_15",
    "SE_17", "SE_29", "SE_32", "SE_34", "SE_39", "SE_40", "SE_45"
]

SE_data = SE_data[selected_SEs]

In [ ]:
SE_data

### Monthly & Daily S/E values

In [ ]:
# Fixed Monthly proportions
monthly_proportions = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# Function to disaggregate yearly values
def disaggregate_yearly_to_monthly(yearly_value, proportions):
    return [yearly_value * proportion for proportion in proportions]

# Apply the function to each relevant column
monthly_data = {}
for column in SE_data.columns:
    if column.startswith('SE'):
        yearly_values = SE_data[column].values
        monthly_values = [disaggregate_yearly_to_monthly(yearly_value, monthly_proportions) for yearly_value in yearly_values]
        monthly_values_flat = [value for sublist in monthly_values for value in sublist]
        monthly_data[column] = monthly_values_flat

# Create a new DataFrame for monthly data
years = SE_data['Year'].values
NUTS_IDs = SE_data['NUTS_ID'].values  # Extract NUTS_ID values
months = list(range(1, 13))

# Expand the Year and NUTS_ID columns for monthly data
monthly_years = [year for year in years for _ in months]
monthly_months = months * len(years)
monthly_nuts_ids = [nuts_id for nuts_id in NUTS_IDs for _ in months]  # Repeat NUTS_ID for each month

# Create the DataFrame
SE_monthly_df = pd.DataFrame(monthly_data)
SE_monthly_df.insert(0, 'NUTS_ID', monthly_nuts_ids)
SE_monthly_df.insert(1, 'Year', monthly_years)
SE_monthly_df.insert(2, 'Month', monthly_months)

# Display the resulting DataFrame
# import ace_tools as tools; tools.display_dataframe_to_user(name="Disaggregated Monthly Data with NUTS_ID", dataframe=SE_monthly_df)

In [ ]:
SE_monthly_df

In [ ]:
# SE_monthly_df[SE_monthly_df["NUTS_ID"] == "TRC34"]

In [ ]:
# Initialize dictionaries and lists for daily data
daily_data = {}
date_list = []
nuts_id_list = []  # List to store repeated NUTS_IDs for daily data

# Generate daily data for each unique NUTS_ID and month
for nuts_id in SE_monthly_df['NUTS_ID'].unique():
    # Filter data for the current NUTS_ID
    nuts_data = SE_monthly_df[SE_monthly_df['NUTS_ID'] == nuts_id]

    for year, month in zip(nuts_data['Year'], nuts_data['Month']):
        # Generate the list of dates for each month for this specific NUTS_ID
        num_days = pd.Period(f'{year}-{month}').days_in_month
        dates = pd.date_range(start=f'{year}-{month}-01', periods=num_days)

        # Append generated dates and NUTS_IDs to lists
        date_list.extend(dates)
        nuts_id_list.extend([nuts_id] * num_days)

        # Generate daily data for each column based on monthly values
        for column in SE_monthly_df.columns[3:]:  # Skip 'Year', 'Month', and 'NUTS_ID'
            if column not in daily_data:
                daily_data[column] = []

            # Get the monthly value and extend it for each day in the month
            monthly_value = nuts_data[(nuts_data['Year'] == year) & (nuts_data['Month'] == month)][column].values[0]
            daily_data[column].extend([monthly_value] * num_days)

# Create the new DataFrame with 'Date' and 'NUTS_ID' columns
SE_daily_df = pd.DataFrame(daily_data)
SE_daily_df.insert(0, 'NUTS_ID', nuts_id_list)
SE_daily_df.insert(1, 'Date', date_list)

In [ ]:
SE_daily_df = SE_daily_df.sort_values(by=["NUTS_ID", "Date"])

In [ ]:
SE_daily_df

### SE Indicators by Watersheds (Station No.)

In [ ]:
WEF_Model_WSHEDS_SA = pd.read_excel('/***/***/StreamflowStationsWSHEDS_SA.xlsx') # Replace with the actual path to the file

In [ ]:
WEF_Model_WSHEDS_SA

In [ ]:
WEF_Model_WSHEDS_A = pd.read_excel('/***/***/StreamflowStationsWSHEDS_SA(km2).xlsx') # Replace with the actual path to the file

In [ ]:
WEF_Model_WSHEDS_A

In [ ]:
WEF_Model_CITIES_A = pd.read_excel('/***/***/Turkish_Cities_Area(km2).xlsx') # Replace with the actual path to the file

In [ ]:
WEF_Model_CITIES_A

In [ ]:
# WEF_Model_WSHEDS_SA = WEF_Model_WSHEDS_SA[~WEF_Model_WSHEDS_SA['Station No.'].isin(stations_to_drop)]

In [ ]:
# Step 1: Melt city-area-in-watershed table to long format
area_long = WEF_Model_WSHEDS_A.melt(
    id_vars='Station No.',
    var_name='NUTS_ID',
    value_name='City_Area_in_Watershed'
)

# Step 2: Merge with city total area
area_long = area_long.merge(
    WEF_Model_CITIES_A[['NUTS_ID', 'Area_calc']],
    on='NUTS_ID',
    how='left'
)
area_long = area_long.rename(columns={'Area_calc': 'City_Total_Area'})

# Step 3: Merge with watershed total area
area_long = area_long.merge(
    WEF_Model_info_data_07[['Station No.', 'Area_calc']],
    on='Station No.',
    how='left'
)
area_long = area_long.rename(columns={'Area_calc': 'Watershed_Total_Area'})

# Step 4: Drop zero or null contributions
area_long = area_long[area_long['City_Area_in_Watershed'] > 0].copy()

# Step 5: Calculate unnormalized share metrics
area_long['City_Share'] = area_long['City_Area_in_Watershed'] / area_long['City_Total_Area']
area_long['Station_Share'] = area_long['City_Area_in_Watershed'] / area_long['Watershed_Total_Area']

# ✅ Step 6: Filter cities that contribute at least 10% to the station
area_long = area_long[area_long['Station_Share'] > 0.10].copy()

# ✅ Step 7: Normalize Station_Share per station (so sum = 1)
area_long['Norm_Station_Share'] = area_long.groupby('Station No.')['Station_Share'].transform(lambda x: x / x.sum())

# ✅ Step 8: Final Adjusted Weight for SE aggregation
area_long['SE_Adj_Weight'] = area_long['City_Share'] * area_long['Norm_Station_Share']

# Reset index for neatness
area_long.reset_index(drop=True, inplace=True)

In [ ]:
area_long

In [ ]:
result_data = []

# Loop through each station and its contributing cities
for station, group in area_long.groupby('Station No.'):
    weighted_se_data = pd.DataFrame()

    for _, row in group.iterrows():
        city_id = row['NUTS_ID']
        weight = row['SE_Adj_Weight']

        # Get SE data for the city
        city_se = SE_daily_df[SE_daily_df['NUTS_ID'] == city_id].copy()
        if city_se.empty:
            continue  # Skip if no SE data for this city

        # Multiply each SE column by the adjusted weight
        weighted = city_se.loc[:, 'SE_06':].multiply(weight)

        # Add 'Date' back
        weighted['Date'] = city_se['Date'].values

        # Accumulate weighted SEs by date
        weighted.set_index('Date', inplace=True)

        if weighted_se_data.empty:
            weighted_se_data = weighted
        else:
            weighted_se_data = weighted_se_data.add(weighted, fill_value=0)

    # Finalize the DataFrame for this station
    weighted_se_data.reset_index(inplace=True)
    weighted_se_data['Station No.'] = station

    # Order columns: Station No., Date, SE values
    ordered_cols = ['Station No.', 'Date'] + [col for col in weighted_se_data.columns if col not in ['Station No.', 'Date']]
    weighted_se_data = weighted_se_data[ordered_cols]

    result_data.append(weighted_se_data)

# Combine all station-level data
WEF_Model_SE_data = pd.concat(result_data, ignore_index=True)
WEF_Model_SE_data.sort_values(by=['Station No.', 'Date'], inplace=True)

In [ ]:
# Select and reorder key columns for clarity
station_city_mapping = area_long[[
    'Station No.',
    'NUTS_ID',
    'City_Area_in_Watershed',
    'City_Total_Area',
    'Watershed_Total_Area',
    'City_Share',
    'Station_Share',
    'Norm_Station_Share',
    'SE_Adj_Weight'
]].copy()

# Optional: round values for readability
station_city_mapping = station_city_mapping.round({
    'City_Area_in_Watershed': 3,
    'City_Total_Area': 3,
    'Watershed_Total_Area': 3,
    'City_Share': 4,
    'Station_Share': 4,
    'Normalized_Station_Share': 4,
    'Adjusted_Weight': 4
})

In [ ]:
station_city_mapping

## save station_city_mapping

In [ ]:
# # Save model_histories to a file
# with open('/***/***/station_city_mapping.pkl', 'wb') as file: # Replace with the actual path to the file
#     pickle.dump(station_city_mapping, file)

In [ ]:
# # Load model_histories from a file
# with open('/***/***/station_city_mapping.pkl', 'rb') as file: # Replace with the actual path to the file
#     station_city_mapping = pickle.load(file)

In [ ]:
# WEF_Model_SE_data[WEF_Model_SE_data["Station No."] == "D07A009"]

# Model Full dataset (Streamflow, S/E indicators, and Climate)

In [ ]:
WEF_Model_streamflow_data.columns

In [ ]:
WEF_Model_streamflow_data

In [ ]:
WEF_Model_Climate_data.columns

In [ ]:
WEF_Model_Climate_data

In [ ]:
WEF_Model_SE_data.columns

In [ ]:
WEF_Model_SE_data

In [ ]:
WEF_Model_streamflow_data = WEF_Model_streamflow_data.copy()
WEF_Model_Climate_data = WEF_Model_Climate_data.copy()
WEF_Model_SE_data = WEF_Model_SE_data.copy()

WEF_Model_streamflow_data['Date'] = pd.to_datetime(WEF_Model_streamflow_data['Date'])
WEF_Model_Climate_data['Date'] = pd.to_datetime(WEF_Model_Climate_data['Date'])
WEF_Model_SE_data['Date'] = pd.to_datetime(WEF_Model_SE_data['Date'])

# Step 1: Merge WEF_Model_streamflow_data and WEF_Model_Climate_data on 'Station No.' and 'Date'
WEF_Model_data = pd.merge(
    WEF_Model_streamflow_data,
    WEF_Model_Climate_data,
    on=['Station No.', 'Date'],
    how='inner'
)

# Step 2: Merge the resulting DataFrame with WEF_Model_SE_data on 'Station No.' and 'Date'
WEF_Model_data = pd.merge(
    WEF_Model_data,
    WEF_Model_SE_data,
    on=['Station No.', 'Date'],
    how='inner'
)

In [ ]:
WEF_Model_data

In [ ]:
# Add a "Month" column based on the "Date" column, right after the "Flow rate (m3/s)" column
WEF_Model_data['Month'] = WEF_Model_data['Date'].dt.month

# Rearrange the columns to place "Month" after "Flow rate (m3/s)"
columns = list(WEF_Model_data.columns)
# Insert "Month" right after "Flow rate (m3/s)"
columns.insert(columns.index("Flow rate (m3/s)") + 1, columns.pop(columns.index("Month")))
WEF_Model_data = WEF_Model_data[columns]

In [ ]:
# Add 'Area' column by merging with WEF_Model_info_data_07
WEF_Model_data = WEF_Model_data.merge(
    WEF_Model_info_data_07[['Station No.', 'Area_calc']],
    on='Station No.',
    how='left'
)

# Rename 'Area_calc' to 'Area' after merging
WEF_Model_data.rename(columns={'Area_calc': 'Area'}, inplace=True)

# Add 'Elevation' column by merging with WEF_Model_info_data_07
WEF_Model_data = WEF_Model_data.merge(
    WEF_Model_info_data_07[['Station No.', 'Elev_calc']],
    on='Station No.',
    how='left'
)

# Rename 'Elev_calc' to 'Elevation' after merging
WEF_Model_data.rename(columns={'Elev_calc': 'Elevation'}, inplace=True)

In [ ]:
WEF_Model_data

In [ ]:
print(WEF_Model_data.columns)

In [ ]:
# WEF_Model_data.to_csv('WEF_Model007_data.csv', index=False)

## Model dataset Analysis

In [ ]:
# Get unique station numbers
WEF_Model_stations = WEF_Model_data['Station No.'].unique()
print(len(WEF_Model_stations))
WEF_Model_stations

In [ ]:
# Group by 'Station No.' and calculate the mean
stations_catchments_char = WEF_Model_data.groupby('Station No.')[WEF_Model_data.select_dtypes(include='number').columns].mean().reset_index()

stations_catchments_char

In [ ]:
# stations_catchments_char.to_excel('stations_catchments_char.xlsx', index=False)

In [ ]:
WEF_Model_data

## Saved Model dataset

In [ ]:
# WEF_Model_data.to_csv('/***/***/TUR Thesis Models (export)/WEF_Model_data.csv', index=False) # Replace with the actual path to the file

In [ ]:
WEF_Model_data = pd.read_csv('/***/***/WEF_Model_data.csv') # Replace with the actual path to the file

In [ ]:
WEF_Model_data['Date'] = pd.to_datetime(WEF_Model_data['Date']) 

In [ ]:
WEF_Model_data

## Training/Testing sets

In [ ]:
# Create a dictionary to store train and test datasets for each station
train_test_sets = {}

# Define the date ranges
train_start_date = datetime(2007, 10, 1)
train_end_date = datetime(2017, 9, 30)
test_start_date = datetime(2017, 10, 1)
test_end_date = datetime(2020, 9, 30)

# Iterate over unique stations
for station in WEF_Model_data['Station No.'].unique():
    # Define the test conditions for the specific station
    test_conditions = (
        (WEF_Model_data['Station No.'] == station) &
        (WEF_Model_data['Date'] >= test_start_date) &
        (WEF_Model_data['Date'] <= test_end_date)
    )

    # Define the train conditions for the specific station
    train_conditions = (
        (WEF_Model_data['Date'] >= train_start_date) &
        (WEF_Model_data['Date'] <= train_end_date)
    )

    # Split the data into training and testing sets
    train_data = WEF_Model_data[train_conditions]
    test_data = WEF_Model_data[test_conditions]

    # Store the results in the dictionary
    train_test_sets[station] = {
        'train': train_data,
        'test': test_data
    }

# Access the train and test sets for a specific station
# example_station = 'D26A055'
# WEF_Model_train_data_D26A055 = train_test_sets[example_station]['train']
# WEF_Model_test_data_D26A055 = train_test_sets[example_station]['test']

## All Stations Model

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf

# 💠 Set random seed for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# # ⚠️ Makes execution slower but deterministic
# os.environ["TF_DETERMINISTIC_OPS"] = "1"

In [ ]:
test_dates = pd.date_range(start=test_start_date, end=test_end_date, freq='D')

In [ ]:
# Group by 'Station No.' and calculate the mean
stations_catchments_char = WEF_Model_data.groupby('Station No.')[WEF_Model_data.select_dtypes(include='number').columns].mean().reset_index()

stations_catchments_char

In [ ]:
import pandas as pd

def get_area_class_bounds(area):
    """Returns fixed area class bounds based on REFORM-style intervals."""
    if area <= 10:
        return 0, 10
    elif area <= 100:
        return 10, 100
    elif area <= 1000:
        return 100, 1000
    elif area <= 10000:
        return 1000, 10000
    else:
        return 10000, float('inf')

def get_elevation_class_bounds(elevation):
    """Returns fixed elevation class bounds based on REFORM-style intervals."""
    if elevation < 200:
        return float('-inf'), 200
    elif elevation < 800:
        return 200, 800
    else:
        return 800, float('inf')

# List to store results
results = []

# Loop over each station in your station_characteristics DataFrame
for idx, station in stations_catchments_char.iterrows():
    station_id = station['Station No.']
    area = station['Area']
    elev = station['Elevation']

    # Logarithmic bounds for area
    try:
        min_area, max_area = get_area_class_bounds(area)
        min_elev, max_elev = get_elevation_class_bounds(elev)
    except ValueError:
        continue  # skip if invalid values

    # Find relevant stations
    filtered = stations_catchments_char[
        (stations_catchments_char['Area'] >= min_area) &
        (stations_catchments_char['Area'] <= max_area) &
        (stations_catchments_char['Elevation'] >= min_elev) &
        (stations_catchments_char['Elevation'] < max_elev)
    ]

    # Store results
    results.append({
        'Station No.': station_id,
        'Relevant Stations count': len(filtered),
        'Relevant Stations': filtered['Station No.'].tolist()
    })

# Convert to summary DataFrame
RelSt_df = pd.DataFrame(results)

# Display the summary
print(RelSt_df)


# Calculate summary statistics
min_count = RelSt_df['Relevant Stations count'].min()
max_count = RelSt_df['Relevant Stations count'].max()
mean_count = RelSt_df['Relevant Stations count'].mean()
mode_count = RelSt_df['Relevant Stations count'].mode().iloc[0]  # most frequent value

# Print the statistics
print("Summary Statistics for Relevant Stations count:")
print(f"Minimum: {min_count}")
print(f"Maximum: {max_count}")
print(f"Mean: {mean_count:.2f}")
print(f"Most Frequent (Mode): {mode_count}")

In [ ]:
import os
import gc
import joblib
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K

window_size = 14

# Define temporal and static features
temporal_feature_names = ['Month',
'u_component_of_wind_10m',
'v_component_of_wind_10m',
'dewpoint_temperature_2m',
'temperature_2m',
'snow_depth',
'snow_density',
'soil_temperature_level_1',
'volumetric_soil_water_layer_1',
'total_precipitation_sum',
'total_evaporation_sum',
'runoff_sum',
'surface_runoff_sum',
'sub_surface_runoff_sum',
'surface_solar_radiation_downwards_sum',
'surface_thermal_radiation_downwards_sum',
'surface_net_solar_radiation_sum',
'surface_net_thermal_radiation_sum',
'surface_latent_heat_flux_sum',
'surface_sensible_heat_flux_sum',
]

static_feature_names = ['Area', 'Elevation']

# Create sliding windows for LSTM
def create_sliding_windows(X, y, window_size):
    X_seq, y_seq = [], []
    for i in range(len(X) - window_size):
        X_seq.append(X[i:i+window_size])
        y_seq.append(y[i+window_size])
    return np.array(X_seq), np.array(y_seq)

# Define model
class TwoBranchHybridModel:
    def __init__(self, num_temporal_features, num_static_features, window_size):
        self.temporal_input_dim = num_temporal_features
        self.static_input_dim = num_static_features
        self.window_size = window_size
        self.model = self._build_model()

    def _build_model(self):

        input_temporal = Input(shape=(self.window_size, self.temporal_input_dim), name="temporal_input")
        x = LSTM(64, return_sequences=True)(input_temporal)
        x = LSTM(32)(x)
        
        input_static = Input(shape=(self.static_input_dim,), name="static_input")
        y = Dense(32, activation='relu')(input_static)
        
        combined = Concatenate()([x, y])
        z = Dense(16, activation='relu')(combined)
        output = Dense(1)(z)
        
        model = Model(inputs={'temporal_input': input_temporal, 'static_input': input_static}, outputs=output)
        model.compile(loss='huber', optimizer=Adam(learning_rate=0.0001), metrics=['mae'])
        
        return model

    def fit(self, X_temporal, X_static, y, **kwargs):
        return self.model.fit({'temporal_input': X_temporal, 'static_input': X_static}, y, **kwargs)

    def predict(self, X_temporal, X_static):
        return self.model.predict({'temporal_input': X_temporal, 'static_input': X_static}).squeeze()

    def summary(self):
        return self.model.summary()

# Load previous models if any
if os.path.exists('/***/***/saved_WEF_Model_histories.joblib'): # Replace with the actual path to the file
    saved_WEF_Model_histories = joblib.load('/***/***/saved_WEF_Model_histories.joblib') # Replace with the actual path to the file
    print("📂 Loaded existing saved_WEF_Model_histories.")
else:
    saved_WEF_Model_histories = {}
    print("📁 No previous saved_WEF_Model_histories found. Starting fresh.")

# Main training loop
for station in WEF_Model_data['Station No.'].unique():
    if station in saved_WEF_Model_histories:
        print(f"\n✅ Skipping station {station} - Already processed. Plotting results.")
        plot_series(test_dates[window_size:],
                    saved_WEF_Model_histories[station]['y_test'][window_size:],
                    saved_WEF_Model_histories[station]['y_pred'],
                    title=f"{station} Model (Previously Trained)")
        continue

    train_data = train_test_sets[station]['train'].copy()
    test_data = train_test_sets[station]['test'].copy()

    station_characteristics = stations_catchments_char[stations_catchments_char['Station No.'] == station]
    if station_characteristics.empty:
        print(f"\n❌ Skipping station {station} - No characteristics found.")
        continue

    relevant_stations_series = RelSt_df.loc[RelSt_df['Station No.'] == station, 'Relevant Stations']
    if relevant_stations_series.empty:
        print(f"\n❌ Skipping station {station} - No relevant stations found for training.")
        continue

    # Extract the actual list from the Series
    relevant_stations = relevant_stations_series.iloc[0]

    print(f"\n✅ Station: {station} - Found {len(relevant_stations)} relevant stations.")

    train_data = train_data[train_data['Station No.'].isin(relevant_stations)]
    test_data = test_data[test_data['Station No.'].isin(relevant_stations)]

    if train_data.empty:
        print(f"\n❌ Skipping station {station} - No training data available after filtering.")
        continue

    X_train = train_data.drop(columns=['Station No.', 'Date', 'Flow rate (m3/s)'])
    y_train = train_data['Flow rate (m3/s)']
    X_test = test_data.drop(columns=['Station No.', 'Date', 'Flow rate (m3/s)'])
    y_test = test_data['Flow rate (m3/s)']

    valid_idx = ~(X_train.isna().any(axis=1) | y_train.isna())
    X_train, y_train = X_train[valid_idx], y_train[valid_idx]
    valid_test_idx = ~(X_test.isna().any(axis=1) | y_test.isna())
    X_test, y_test = X_test[valid_test_idx], y_test[valid_test_idx]

    X_train_static = X_train[static_feature_names]
    X_train_temporal = X_train[temporal_feature_names]
    X_test_static = X_test[static_feature_names]
    X_test_temporal = X_test[temporal_feature_names]

    # Scale and reshape
    scaler_static = StandardScaler()
    scaler_temporal = StandardScaler()
    X_train_static_scaled = scaler_static.fit_transform(X_train_static)
    X_test_static_scaled = scaler_static.transform(X_test_static)
    X_train_temporal_scaled = scaler_temporal.fit_transform(X_train_temporal)
    X_test_temporal_scaled = scaler_temporal.transform(X_test_temporal)

    X_train_temporal_seq, y_train_seq = create_sliding_windows(X_train_temporal_scaled, y_train.values, window_size)
    X_test_temporal_seq, y_test_seq = create_sliding_windows(X_test_temporal_scaled, y_test.values, window_size)

    X_train_static_seq = X_train_static_scaled[window_size:]
    X_test_static_seq = X_test_static_scaled[window_size:]

    model = TwoBranchHybridModel(
        num_temporal_features=len(temporal_feature_names),
        num_static_features=len(static_feature_names),
        window_size=window_size
    )

    model.fit(X_train_temporal_seq, X_train_static_seq, y_train_seq, epochs=100, batch_size=128, verbose=1)
    y_pred = model.predict(X_test_temporal_seq, X_test_static_seq)

    saved_WEF_Model_histories[station] = {
        'model': model,
        'scaler_static': scaler_static,
        'scaler_temporal': scaler_temporal,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'y_pred': y_pred,
        'relevant_stations': relevant_stations,
    }

    joblib.dump(saved_WEF_Model_histories, '/***/***/saved_WEF_Model_histories.joblib') # Replace with the actual path to the file
    print(f"💾 Saved {station} to saved_WEF_Model_histories.joblib")

    plot_series(test_dates[window_size:], y_test_seq, y_pred, title=f"{station} Model")

    del X_train_static, X_train_temporal, X_test_static, X_test_temporal
    del X_train_static_scaled, X_train_temporal_scaled
    del X_test_static_scaled, X_test_temporal_scaled
    K.clear_session()
    gc.collect()

print("✅ Completed training and saving all stations to 'saved_WEF_Model_histories.joblib'")